# Week6: Class

Pytorch의 Dataset 추상클래스를 상속받아 CustomDataset 만들어보기!

- 현재는 X_data, y_data를 임의로 선정해두었지만, 원한다면 어떠한 데이터를 쓰셔도 상관없습니다. 

- 선언되어있는 3가지 메서드는 모두 구현하셔야 합니다. 
- 연습을 위한 추가적인 메서드들은 언제나 환영합니다!



---
# Preface
많은 양의 data를 이용해서 딥러닝 모델을 학습시키는 일이 많아지면서 그 많은 양의 data를 한번에 불러오려면 시간이 오래걸리는 것을 넘어서서 data의 크기가 RAM용량을 넘을경우 메모리 관리에 오버헤드가 커져 비효율적인 학습이 이뤄질 수 있다. 따라서 데이터를 한번에 다 부르지 않고 하나씩만 불러서 쓰는 방식을 택하면 쾌적하게 모델을 돌릴 수 있다. 그래서 모든 데이터를 한번에 불러놓고 쓰는 기존의 dataset말고 custom dataset을 만들어야할 필요가 있다. 또한 길이가 변하는 input에 대해서 batch를 만들기 위해서는 dataloader에서 batch를 만드는 부분을 수정해야할 필요가 있어 custom dataloader를 사용해야 한다. 파이토치에서는 데이터를 좀 더 쉽게 다룰 수 있도록 유용한 도구로서 데이터셋(`Dataset`)과 데이터로더(`DataLoader`)를 제공한다. 이를 사용하면 미니 배치 학습, 데이터 셔플(shuffle), 병렬 처리까지 간단히 수행할 수 있습니다. 기본적인 사용 방법은 `Dataset`을 정의하고, 이를 `DataLoader`에 전달하는 것이다.

커스텀 데이터셋을 만들 때, 일단 가장 기본적인 뼈대는 아래와 같다. 여기서 필수적으로 `overriding`해야 하는 메소드는 3개다.
```
class CustomDataset(torch.utils.data.Dataset): 
  def __init__(self):
  # 데이터셋의 전처리를 해주는 부분
  
  def __len__(self):
  # 데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분
  # len(dataset)을 했을 때 데이터셋의 크기를 리턴하는 magic method
  
  def __getitem__(self, idx): 
  # 데이터셋에서 특정 1개의 샘플을 가져오는 함수
  # dataset[i]을 했을 때 i번째 샘플을 가져오도록 하는 인덱싱을 위한 magic method
```

## 들어 가기에 앞서: Change betwwen numpy.ndarray and torch.Tensor
현재까지 딥러닝은 도메인에 적합한 여러 아키텍쳐가 나왔다. 하지만 데이터도 많고 모델이 큰 만큼 파라미터도 많기 때문에 최적화된 행렬(텐서) 연산을 사용한다. 따라서 데이터도 행렬처럼 표현이 가능해야 한다. 이때 주로 많이 사용하는 것이 `numpy` 라이브러리의 `ndarray` 클래스다. 한편, `torch`를 이용한 딥러닝에서 행렬 연산에 최적화되어 사용되는 실제 텐서는  `Tensor` 클래스다. 따라서 필요에 따라 둘 사이의 자유로운 변환을 할 줄 알아야 한다.

### Numpy에서 Tensor로: torch.Tensor() vs torch.from_numpy()
- torch.Tensor() 는 Numpy array의 사본일 뿐이다. 그래서 tensor의 값을 변경하더라도 Numpy array자체의 값이 달라지지 않는다. 하지만 torch.from_numpy()는 자동으로 input array의 dtype을 상속받고 tensor와 메모리 버퍼를 공유하기 때문에 tensor의 값이 변경되면 Numpy array값이 변경된다. 예시를 통해 알아보자.

##### 1) torch.Tensor()

tensor로 변환할 때 새 메모리를 할당한다.

In [16]:
import torch
import numpy as np

a=np.array([1,2,3,4])
b=torch.Tensor(a)
print('output tensor:',b)

b[0]=-1
print('Tensor is changed:', b)
print('But it cannot change np.array:',a)

output tensor: tensor([1., 2., 3., 4.])
Tensor is changed: tensor([-1.,  2.,  3.,  4.])
But it cannot change np.array: [1 2 3 4]


##### 2) torch.from_numpy()

tensor로 변환할 때, 원래 메모리를 상속받는다. (=as_tensor())

In [17]:
a=np.array([1,2,3,4])
b=torch.from_numpy(a)
print('output tensor:',b)

b[0]=-1
print('Tensor is changed:', b)
print('It can change np.array:',a)

output tensor: tensor([1, 2, 3, 4])
Tensor is changed: tensor([-1,  2,  3,  4])
It can change np.array: [-1  2  3  4]


### Tensor에서 Numpy로: numpy()
- 반대로 Tensor를 Numpy array로 바꾸고 싶다면 numpy()함수를 사용하면 된다. 사용법은 아래와 같다.

In [21]:
import torch
import numpy as np

a = torch.rand(3,3)
b = a.numpy()
display(type(b))
display(b)

numpy.ndarray

array([[0.8153333 , 0.53005856, 0.8592659 ],
       [0.5831757 , 0.6514638 , 0.2412979 ],
       [0.5439898 , 0.8240437 , 0.7072638 ]], dtype=float32)

이외에 `torch.Tensor`와 `torch.tesnor()`의 차이, `torch.Tensor`와 `torch.autograd.Variable`의 차이, `Tensor`의 자료형, `Tensor`와 관련된 함수들 또는 메소드는 다음의 링크를 참고하라: https://subinium.github.io/pytorch-Tensor-Variable/

# Week 6 Class Homework

In [23]:
from torch.utils.data import Dataset
import numpy as np

In [24]:
X_data = np.random.rand(10,10)
y_data = np.ones(10)

## Make CustomDataset with Overriding

In [176]:
class CustomDataset(Dataset): 
    def __init__(self, X_data, y_data):
        # parent class 초기화. 범용성을 위해 python2 방식을 사용.
        super(CustomDataset, self).__init__()
        
        self.X_data = X_data
        self.y_data = y_data
        
    # 총 데이터의 개수를 리턴
    def __len__(self):
        return len(self.X_data)

    # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 PyTorch의 Tensor 형태로 리턴
    def __getitem__(self, idx): 
        # y_data에서 인덱스를 이용해서 하나의 데이터만 뽑으면 scalar이다.
        # 이를 Tensor라는 클래스 생성자에 바로 전달하니 __getitem__호출 시 다음과 같은 에러가 발생한다.
        # TypeError: new(): data must be a sequence (got numpy.float64)
        # Tensor의 argument로 scalar가 아닌 seqeunce를 달라는 것 같은데, 
        # Tensor(1) 이런 식으로 scalar로 Tensor 생성자에 전달하면 1이 아닌 다른 값을 내뱉는다.
        # 어떤 값인지는 알아 봐야겠지만, 안전하게 torch.tensor()함수를 사용하는 게 좋은 것 같다.
        return torch.tensor(self.X_data[idx]), torch.tensor(self.y_data[idx])

In [177]:
dataset = CustomDataset(X_data, y_data)

In [178]:
# 실제 데이터 X_data가 instance attribute인 X_data에 잘 들어 간 것을 확인할 수 있다.
X_data == dataset.X_data

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])

In [179]:
y_data == dataset.y_data

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [180]:
# CustomDataset의 길이가 잘 출력되는 것을 확인할 수 있다.
len(dataset)

10

In [184]:
# 실제 데이터의 첫번째 값들
X_data[0], y_data[0]

(array([0.31682092, 0.35788936, 0.04984044, 0.35053424, 0.69713628,
        0.40714328, 0.92830629, 0.24683282, 0.67307919, 0.90673534]),
 1.0)

In [182]:
# 데이터의 텐서 값. 소수점이 생략된 것 빼고 동일하다.
dataset[0]

(tensor([0.3168, 0.3579, 0.0498, 0.3505, 0.6971, 0.4071, 0.9283, 0.2468, 0.6731,
         0.9067], dtype=torch.float64),
 tensor(1., dtype=torch.float64))

# Application to Iris classifier using PyTorch

In [191]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()

iris_df = pd.DataFrame(data=np.c_[iris['data'], iris['target']], columns=iris['feature_names']+['target'])
iris_df['target'] = iris_df['target'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})

`sklearn.datasets`의 `load_iris`함수를 이용하면 iris와 관련된 데이터들을 딕셔너리와 유사한 타입(실제로는 `sklearn.utils.Bunch`)으로 반환한다. 이때 `'data'`와 `'target'`은 `numpy.ndarry` type이므로 이를 이용해서 텐서로 만들어 간단한 분류 모델을 만들어 보자.

In [197]:
print(type(iris['data']))
print(type(iris['target']))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [198]:
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [286]:
from torch.utils.data import Dataset

class IrisDataset(Dataset):
    def __init__(self, X_data, y_data):
        super(IrisDataset, self).__init__()
        self.X_data = X_data
        self.y_data = y_data
        # model parameter가 float32이기때문에 input도 dtype을 통일해야한다.
        self.X_tensor = torch.tensor(X_data, dtype=torch.float32)
        self.y_tensor = torch.tensor(y_data) #.reshape(-1,1))
        # one-hot encoding을 하니 multi-target이라고 되려 성질을 낸다. 과한 친절이었나 보다.
#         self.y_tensor_one_hot = torch.nn.functional.one_hot(self.y_tensor, num_classes=3)
#         self.y_tensor_one_hot = torch.as_tensor(self.y_tensor_one_hot, dtype=torch.bool)
    
    def __len__(self):
        return len(self.X_data)
    
    def __getitem__(self, idx):
        return self.X_tensor[idx], self.y_tensor[idx]

In [287]:
dataset = IrisDataset(iris['data'], iris['target'])

In [288]:
print(dataset[0][0].dtype)
print(dataset[0][1].dtype)

torch.float32
torch.int64


In [289]:
dataset[0:2]

(tensor([[5.1000, 3.5000, 1.4000, 0.2000],
         [4.9000, 3.0000, 1.4000, 0.2000]]),
 tensor([0, 0]))

In [290]:
display(iris['data'][0], iris['target'][0])

array([5.1, 3.5, 1.4, 0.2])

0

In [308]:
class MLPclassifier(torch.nn.Module):
    def __init__(self, num_input, H1, H2, num_classes):
        super(MLPclassifier, self).__init__()
        self.linear1 = torch.nn.Linear(num_input, H1)
        self.linear2 = torch.nn.Linear(H1, H2)
        self.linear3 = torch.nn.Linear(H2, num_classes)
        
    def forward(self, x):
        x = torch.nn.functional.relu(self.linear1(x))
        x = torch.nn.functional.relu(self.linear2(x))
        x = self.linear3(x)
        return x

In [311]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
model = MLPclassifier(len(dataset[0][0]), 20, 10, 3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

In [312]:
nb_epochs = 100
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        # print(batch_idx)
        # print(samples)
        X_train, y_train = samples
        X_train.float() # RuntimeError: expected scalar type Float but found Double
        
        # H(x) 계산
        X_pred = model(X_train)
        
        # loss 계산
        # loss = torch.nn.CrossEntropyLoss(X_pred, y_train) 이런 식으로 한 줄로 작성하면 다음과 같은 에러 발생
        # runtimeerror: boolean value of tensor with more than one value is ambiguous
        criterion_label = torch.nn.CrossEntropyLoss()
        loss = criterion_label(X_pred, y_train)
        
        # bp
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f'Epoch: {epoch}/{nb_epochs}, Batch: {batch_idx+1}/{len(dataloader)}, Loss: {round(loss.item(), 5)}')

Epoch: 0/100, Batch: 1/5, Loss: 1.15739
Epoch: 0/100, Batch: 2/5, Loss: 1.15915
Epoch: 0/100, Batch: 3/5, Loss: 1.14364
Epoch: 0/100, Batch: 4/5, Loss: 1.08853
Epoch: 0/100, Batch: 5/5, Loss: 1.15817
Epoch: 1/100, Batch: 1/5, Loss: 1.13725
Epoch: 1/100, Batch: 2/5, Loss: 1.1217
Epoch: 1/100, Batch: 3/5, Loss: 1.06407
Epoch: 1/100, Batch: 4/5, Loss: 1.12542
Epoch: 1/100, Batch: 5/5, Loss: 1.15151
Epoch: 2/100, Batch: 1/5, Loss: 1.13238
Epoch: 2/100, Batch: 2/5, Loss: 1.07511
Epoch: 2/100, Batch: 3/5, Loss: 1.08856
Epoch: 2/100, Batch: 4/5, Loss: 1.10477
Epoch: 2/100, Batch: 5/5, Loss: 1.10904
Epoch: 3/100, Batch: 1/5, Loss: 1.09208
Epoch: 3/100, Batch: 2/5, Loss: 1.08855
Epoch: 3/100, Batch: 3/5, Loss: 1.08984
Epoch: 3/100, Batch: 4/5, Loss: 1.09145
Epoch: 3/100, Batch: 5/5, Loss: 1.06584
Epoch: 4/100, Batch: 1/5, Loss: 1.07487
Epoch: 4/100, Batch: 2/5, Loss: 1.0881
Epoch: 4/100, Batch: 3/5, Loss: 1.06815
Epoch: 4/100, Batch: 4/5, Loss: 1.06457
Epoch: 4/100, Batch: 5/5, Loss: 1.0744
Epo

In [320]:
num_correct = 0
for batch_idx, samples in enumerate(dataloader):
    X, y = samples
    X.float()
    
    X_pred = model(X)
    
    X_pred = torch.argmax(X_pred, dim=1)
    
    num_correct += sum(X_pred == y).numpy()

In [322]:
accuracy = num_correct/len(dataset)
print("Accuracy in all data: ", round(accuracy,3))

Accuracy in all data:  0.987


모델이 잘 학습된 것을 확인할 수 있다.

Q. PyTorch에서 제공하는 데이터셋(e.g., CIFAR, MNIST, STL10)에는 transforms가 적용되는데 커스텀데이터셋에는 그런 어떻게 적용하나? `__init__`에서 따로 구현? 지원하는 함수가 따로 있을 것 같은데...